XMLで記述されているFreeplaneのmmファイルを解析して、VSCODE-mindmapが記述されているjsonファイルで出力<br>
多分FreeMindも対応

In [1]:
import xml.etree.ElementTree as ET
import json
import pprint
import os, sys

In [2]:
fname = 'test'
tree = ET.parse(f'{fname}.mm', parser=ET.XMLParser(encoding='utf-8'))
root = tree.getroot()

In [4]:
# XMLデータを解析する関数
def parse_xml(element):
    parsed_data = {}

    #ノードの情報の処理
    parsed_data[element.tag] = {}
    for item in element.items():
        parsed_data[element.tag][item[0]] = item[1]

    if element:
        #子要素に再帰処理
        for child in element:
            child_data = parse_xml(child)
            if child.tag in parsed_data[element.tag]:
                if isinstance(parsed_data[element.tag][child.tag], list):
                    parsed_data[element.tag][child.tag].append(child_data)
                else:
                    parsed_data[element.tag][child.tag] = [parsed_data[element.tag][child.tag], child_data]
            else:
                parsed_data[element.tag][child.tag] = child_data
                
    return parsed_data


In [5]:
# 解析したデータを辞書データに格納
parsed_data = parse_xml(root)

In [6]:
# Freeplaneの構造をVSCODE-mindmapの構造に変換する関数群
def get_data(old_node_):
    data_ = {
        'id': old_node_['ID'],
        'created': int(old_node_['CREATED']),
        'text': old_node_['TEXT']
    }
    return data_

def get_children_impl(node_):
    child_data = get_data(node_)
    grand_children = get_children(node_)
    return {'data': child_data, 'children':grand_children}
    

def get_children(old_node_):
    children_ = []
    if 'node' not in old_node_.keys():
        return children_
    
    if isinstance(old_node_['node'], dict):
        # 子要素が1つの場合
        children_.append(get_children_impl(old_node_['node']['node']))
    else :
        # 子要素が複数の場合
        for n_ in old_node_['node']:
            if ('node' not in n_.keys()) or ('ID' not in n_['node'].keys()):
                continue
            children_.append(get_children_impl(n_['node']))
    return children_


In [12]:
# 解析したFreeplaneの辞書データ型をVSCODE-mindmapの構造に変換
old_root = parsed_data['map']['node']

new_root = {}
new_root['root'] = {}

new_root['root']['data'] = get_data(old_root['node'])
new_root['root']['children'] = get_children(old_root['node'])


In [14]:
# ファイル出力
with open(f'{fname}.km', 'w') as f:
    json.dump(new_root, f)